In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from stargazer.stargazer import Stargazer
%matplotlib inline
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.api import add_constant
import json
import geopandas as gpd
import plotly.graph_objects as go
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv("data/df_final.csv")

In [3]:
df["Niederlassung"]= df["Niederlassung"].replace("Baden-Würtenberg", "Baden-Wuertenberg")


In [4]:
df.head()

,Unnamed: 0,Alter,Einkommen,Geschlecht,Zeit,KundeNr,Preis,Niederlassung
0,0,30,39919,0.0,29,K1814,NaN,Bayern
1,1,41,53430,0.0,27,K1544,NaN,Baden-Wuerttemberg
2,2,64,73163,0.0,32,K1945,NaN,Baden-Wuerttemberg
3,3,41,36761,0.0,44,K1825,NaN,Baden-Wuerttemberg
4,4,35,42684,1.0,33,K1144,NaN,Nordrhein-Westfalen


<font size = "8"> Abschlusspraesentation </font>

</i> Einfuehrung in Python fuer Daten Analytics</i>

<i>HWR Berlin, Wintersemester 2023-2024</i>

<i>Goekan Goerer und David Kryzanovskij </i>

# 1. Wie viele Autos wurden verkauft?

In [5]:
Anzahl_Autos = df["Preis"].notna().sum()
Anzahl_Autos

1104

# 2. Was ist der Höchst-, Mindest- und Durchschnittspreis der verkauften Autos?

In [6]:
Hoechst_Preis = df["Preis"].max()
print("MAX:", Hoechst_Preis )

Mindest_Preis = df["Preis"].min()
print("Min:", Mindest_Preis)

AVG_Preis = df["Preis"].mean()
print("AVG:", AVG_Preis )

MAX: 165482.31
Min: 21471.65
AVG: 88975.19871376811


# 3. Wie hoch war der Gesamtumsatz?

In [7]:
Gesamt_Umsatz = df["Preis"].sum()
print("Gesamt:", Gesamt_Umsatz)

Gesamt: 98228619.38


# 4. Wie viele Autos wurden pro Bundesland verkauft?

In [8]:
autos_pro_bundesland = df.groupby('Niederlassung')['Preis'].count()
autos_pro_bundesland

Niederlassung
Baden-Wuerttemberg     205
Bayern                 188
Berlin                 126
Brandenburg             37
Düsseldorf               1
Hamburg                 43
Hessen                  96
Niedersachsen           96
Nordrhein-Westfalen    174
Sachsen                 89
Thueringen              49
Name: Preis, dtype: int64

In [9]:
germany_map = gpd.read_file("data/polygons.json")
germany_map.head()

DriverError: Failed to read GeoJSON data

In [ ]:
germany_map = germany_map.rename(columns={"NAME_1":"id"})
germany_map.head()

In [ ]:
germany_map = germany_map.loc[:, ["id", "geometry"]]
germany_map.head()

In [ ]:
data = {
    '': ['Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Düsseldorf', 'Hamburg', 'Hessen', 'Niedersachsen', 'Nordrhein-Westfalen', 'Sachsen', 'Thüringen'],
    'Anzahl der Verkäufe': [205, 188, 126, 37, 1, 43, 96, 96, 174, 89, 49]
}
germany_info = pd.DataFrame(data)
germany_info

In [ ]:
germany_map_merged = pd.merge(germany_map, germany_info)
germany_map_merged

In [ ]:
fig, ax = plt.subplots(1, figsize = (10, 6))
germany_map_merged.plot(column = "Anzahl der Verkäufe", ax = ax,
                        cmap = "Greens", legend = True)

ax.axis("off")
ax.set_title("Anzahl verkaufter Modelle nach Bundesland")
plt.show();

# 5. Wie hoch war der durchschnittliche Umsatz pro Bundesland?

In [ ]:
auto_preis_pro_bundesland = df.groupby('Niederlassung')['Preis'].mean()
auto_preis_pro_bundesland

In [ ]:
germany_map = gpd.read_file("data/polygons.json")

In [ ]:
germany_map = germany_map.rename(columns={"NAME_1":"id"})

In [ ]:
germany_map = germany_map.loc[:, ["id", "geometry"]]

In [ ]:
data_niederlassung = {
    'id': ['Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Düsseldorf', 'Hamburg', 'Hessen', 'Niedersachsen', 'Nordrhein-Westfalen', 'Sachsen', 'Thüringen'],
    'Preis': [90969.560000, 89344.313245, 89584.984048, 86673.640811, 120215.500000, 85549.499070, 85864.929479, 87963.312500, 88451.758103, 89455.080674, 90816.994082]
}
germany_info = pd.DataFrame(data_niederlassung)
germany_info

In [ ]:
germany_map_merged1 = pd.merge(germany_map, germany_info)
germany_map_merged1

In [ ]:
fig, ax = plt.subplots(1, figsize = (10, 6))
germany_map_merged1.plot(column = "Preis", ax = ax,
                        cmap = "Blues", legend = True)

ax.axis("off")
ax.set_title("Durchschnittlicher Umsatz pro verkaufter Einheit nach Bundesland in EUR")
plt.show();

In [ ]:
# 3 key findinds: in thüringen verkaufen wir wenig aber für viel (mehr in thüringen investieren)
 # baden würtemberg läuft sehr gut(verkaufen viel für viel)
    # kein geschäft in Sachsen-Anhalt

# 6. Haben mehr Frauen oder mehr Maenner unsere Autos gekauft?

In [ ]:
anzahl_frauen = df[df['Geschlecht'] == 0]['Preis'].count()
anzahl_maenner = df[df['Geschlecht'] == 1]['Preis'].count()

if anzahl_frauen > anzahl_maenner: 
    print("Die Anzahl der Frauen war hoeher:", anzahl_frauen, ", die Anzahl der Maenner dagegen war", anzahl_maenner)
else: 
    print("Die Anzahl der Maenner war hoeher:", anzahl_maenner, ", die Anzahl der Frauen dagegen war", anzahl_frauen)

In [ ]:
Gekaufte_Autos = ['Maenner', 'Frauen']
 
data = [706, 398]

def func(pct, allvalues):
    absolute = int(pct / 100.*np.sum(allvalues))
    return "{:.1f}%\n({:d} )".format(pct, absolute)

fig, ax = plt.subplots(figsize =(10, 7))
wedges, texts, autotexts = ax.pie(data, 
                                  autopct = lambda pct: func(pct, data), 
                                  labels = Gekaufte_Autos,
                                  shadow = True,
                                  startangle = 90,
                                  textprops = dict(color ="black"))

ax.legend(
          title ="Gekaufte_Autos",
          loc ="center left",
          bbox_to_anchor =(1, 0, 0.5, 1))

ax.set_title('Haben mehr Frauen oder mehr Männer unsere Autos gekauft?')
 
plt.show()

# 7. Wie hoch ist das Durchschnittsalter unserer Kunden?

In [ ]:
durchschnittsalter = df[df['Preis'].isna()]['Alter'].mean()
print(durchschnittsalter)

# 8. Wie hoch ist das Durchschnittsalter der Besucher in unseren Showrooms?

In [ ]:
durchschnittsalter = df[df['Preis'].notna()]['Alter'].mean()
print(durchschnittsalter)

# 9. Was ist das Durchschnittsalter unserer männlichen Kunden?

In [ ]:
anzahl_maenner = df[df['Geschlecht'] == 1]

durchschnittsalter = anzahl_maenner[anzahl_maenner['Preis'].isna()]['Alter'].mean()
print(durchschnittsalter)

# 10. Wie hoch ist das Durchschnittseinkommen unserer Kunden?

In [ ]:
Einkommen = df[df['Preis'].isna()]["Einkommen"].mean()
print("Durchschnittseinkommen:", Einkommen)

# 11. Wie hoch ist die Korrelation (Pearson-Korrelation) zwischen den Variablen Alter, Einkommen, Preis und Zeit? (nur für die Kunden)

In [ ]:
selected_columns = ["Alter", "Einkommen", "Preis", "Zeit"]

new_df = df[selected_columns]

In [ ]:
plt.figure(figsize = (5, 5))

sns.heatmap(
    new_df.select_dtypes(np.number).corr(),
    cmap = 'coolwarm',
    annot = True,
    annot_kws = {"size": 8}
);
plt.show()

# 12. Wie ist die Variable Zeit verteilt? (Kunden und Besucher zusammen)

In [ ]:
sns.distplot(df["Zeit"])

# 13. Wie viele Kunden haben keinen Kredit bei der Bank genommen, um das Auto zu kaufen? Die Kundenabteilung hat vergessen, diese Informationen zu sammeln. Wir können davon ausgehen, dass die Kunden mit einem Jahreseinkommen, das höher als der Autopreis ist, keinen Bankkredit benötigten

In [ ]:
Anzahl_Kreditnehmer = (df['Preis'] > df['Einkommen']).sum()
Anzahl_Kreditnehmer

# 14. Welches sind die sozioökonomischen Merkmale der Kunden, die den Kaufpreis beeinflussen? Wählen Sie die geeigneten abhängigen Variablen aus und schätzen Sie eine Regression unter Verwendung der geeigneten Methode

In [ ]:
model = smf.ols("Preis ~ Alter + Geschlecht + Einkommen", data=df).fit()
Stargazer([model])

In [ ]:
model = sm.OLS.from_formula("Preis ~ Alter + Geschlecht + Einkommen", data=df).fit()
model.summary()

# 15. Prognostizieren Sie den Kaufpreis eines unserer Autos für einen männlichen Kunden im Alter von 32 Jahren mit einem Einkommen von 30.000 Euro. Prognostizieren Sie den Kaufpreis eines unserer Autos für einen männlichen Kunden im Alter von 51 Jahren und mit einem Einkommen von 54.000 Euro

In [ ]:
from statsmodels.api import add_constant

In [ ]:
X_predict = pd.DataFrame({
    'Alter': [32, 51],           
    'Geschlecht': [1, 1],
    'Einkommen': [30000, 54000] 
})

X_predict = add_constant(X_predict, has_constant='add')
X_predict

In [ ]:
predicted_preis = model.predict(X_predict)
predicted_preis

# 16. In Bezug auf die vorherige Frage: Welche Variable beeinflusst den Preis des Autos am meisten? Mit anderen Worten: Die von Ihnen geschätzten Regressionskoeffizienten müssen direkt vergleichbar sein. Wie sollen die Daten transformiert werden? Tipp: Beta-Werte.

In [ ]:
scaler_X = StandardScaler()

In [ ]:
df[['Alter', 'Geschlecht', 'Einkommen']] = scaler_X.fit_transform(df[['Alter', 'Geschlecht', 'Einkommen']])

In [ ]:
scaler_y = StandardScaler()

In [ ]:
df['Preis'] = scaler_y.fit_transform(df[['Preis']])

In [ ]:
try:
    model = sm.OLS.from_formula("Preis ~ Alter + Geschlecht + Einkommen", data=df).fit()
    print(model.summary())
except Exception as e:
    print(f"Error: {str(e)}")

# 17. Schätzen Sie eine Regression, die die Wahrscheinlichkeit des Kaufs eines Autos ermittelt. Verwenden Sie die entsprechende Methode. Tipp: Logistische Regression.

In [ ]:
df = pd.read_csv("data/df_final.csv")
df['gekauft_1'] = df['Preis'].notna().astype(int)

In [ ]:
df.tail()

In [ ]:
reg1_gekauft = smf.logit('gekauft_1 ~ Alter + Geschlecht + Einkommen + Zeit', data=df).fit()

In [ ]:
reg1_gekauft.summary()

In [ ]:
Stargazer([reg1_gekauft])

# 18. Wie hoch ist die Wahrscheinlichkeit, dass ein 32-jähriger männlicher Kunde mit einem Einkommen von 30.000 Euro, der das Auto 30 Minuten lang getestet hat, eines unserer Modelle kauft? Wie hoch ist die Wahrscheinlichkeit, dass ein 51-jähriger männlicher Kunde mit einem Einkommen von 54.000 Euro, der das Auto 45 Minuten lang getestet hat, eines unserer Modelle kauft?

In [ ]:
Y_predict = pd.DataFrame({
    'Alter': [32, 51],           
    'Geschlecht': [1, 1],
    'Einkommen': [30000, 54000],
    "Zeit": [30, 45]
})

Y_predict = add_constant(Y_predict, has_constant='add')
Y_predict

In [ ]:
predicted_preis = reg1_gekauft.predict(Y_predict)
(predicted_preis*100)

# 19. Auf welche Probleme sind Sie bei der Zusammenführung des Datensatzes gestoßen? Stellen Sie die Operationen vor, die Sie zum Zusammenführen und Bereinigen der Daten durchgeführt haben.

geo_df = pd.read_csv("data/geo.txt", sep = "\t")
kunden_df["Einkommen"] = kunden_df["Einkommen"].astype("int")
geo_df["Niederlassung"] = geo_df["Niederlassung"].replace(["Berlin-Charlottenburg", "Berlin-Mitte", "BERLIN"], "Berlin")
data_df.loc[data_df['KundeNr'].isin(["K0701", "K1021"]), 'Einkommen'] = int(data_df['Einkommen'].median())

# 20. Welche Vorschläge würden Sie der Kundenabteilung für die Umfrage im nächsten Jahr machen? Welche zusätzlichen Informationen sollten gesammelt werden? Formulieren Sie zwei Vorschläge.

Wir würden unserer Kundenabteilung vorschlagen in Zukunft Informationen, über den hoechsten abgeschlossenen Bildungsgrad unserer Kunden, als auch das Datum des Kaufes zu sammeln. 